In [1]:
import pandas as pd
from langdetect import detect
import string
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
from mlflow import pyfunc
import requests
import boto3
import json

import nltk
from nltk import word_tokenize 
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('wordnet')

pd.set_option('display.max_rows',500)

/Users/dlite/projects/ml_examples/covid19-repo-recommender/covid19-repo-recommender/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/dlite/projects/ml_examples/covid19-repo-recommender/covid19-repo-recommender/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Sequence, defaultdict
/Users/dlite/projects/ml_examples/covid19-repo-recommender/covid19-repo-recommender/lib/python3.7/site-packages/nltk/lm/vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import 

In [2]:
########################################################
### Import Dataset
########################################################

In [3]:
raw_github_data = pd.read_csv('2020-04-06.tsv', sep='\t', header=0)

In [4]:
########################################################
### Clean Dataset
########################################################

In [5]:
# Filtering down to repos that are likely needing contributors based on past behavior
raw_github_data_filtered = raw_github_data[(raw_github_data['has_merged_prs'] == True) &
    (raw_github_data['has_readme'] == True) &
    (pd.isna(raw_github_data['repo_description']) == False) &
    (pd.isna(raw_github_data['primary_language_name']) == False) &
    (raw_github_data['count_distinct_contributors'] >=2)
]

In [6]:
# Detect language with error handling
def detect_with_error_handle(x):
    try:
        return detect(x)
    except:
        return 'Error'
    
# Check for only latin characters
def has_only_latin_letters(text):
    char_set = string.printable + '—'
    return all((True if x in char_set else False for x in text))

# Remove punctuation
def remove_punctuation(text):
    punctuation_list = string.punctuation + '—'
    return text.translate(str.maketrans('', '', punctuation_list))

In [7]:
## Full set of text processing

# check language, limit to english, and limit repo's with latin characters. Emojis are converted in the process
raw_github_data_filtered['language'] = raw_github_data_filtered['repo_description'].apply(lambda x: 'None' if pd.isna(x) else detect_with_error_handle(str(x)))
raw_github_data_filtered = raw_github_data_filtered[raw_github_data_filtered['language'] == 'en'].copy()
raw_github_data_filtered['is_latin_only_characters'] = raw_github_data_filtered['repo_description'].apply(lambda x: has_only_latin_letters(emoji.demojize(x)))
raw_github_data_filtered = raw_github_data_filtered[raw_github_data_filtered['is_latin_only_characters'] == True].copy()

# clean up repo description, topic, and language, combine into one big bag o' words
raw_github_data_filtered['repo_description_cleaned'] = raw_github_data_filtered['repo_description'].apply(lambda x: remove_punctuation(x))
raw_github_data_filtered['topics'] = raw_github_data_filtered.apply(lambda x: remove_punctuation(str(x['topics']).replace(',','').replace('nan','')), axis=1)
raw_github_data_filtered['topics'].fillna('', inplace=True)
raw_github_data_filtered['description_plus_topics'] = raw_github_data_filtered['repo_description_cleaned']+' '+raw_github_data_filtered['topics']+' '+raw_github_data_filtered['primary_language_name']
raw_github_data_filtered.reset_index(drop=True, inplace=True)

repo_lookup = raw_github_data_filtered

/Users/dlite/projects/ml_examples/covid19-repo-recommender/covid19-repo-recommender/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [8]:
########################################################
### Tokenize
########################################################

In [9]:
# Create class to be used by tokenizer to lemmatize... which change matches words to their roots
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [10]:
# Create a list of stop words that should be removed before tokenizing
stopwords = list(ENGLISH_STOP_WORDS) + ['covid19','coronavirus','virus','corona','covid','pandemic','sarscov2','outbreak','19','disease','2019','2019ncov','cord19','repository','repo','2020','20','covid2019','covidvirus', 'cases','case']

# Create vectorizor of n-grams using stop words and lemmatizer
word_vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word',stop_words=stopwords, tokenizer=LemmaTokenizer())

# Fit vectorizer on existing list of repos and create sparse matrix
sparse_vector_matrix = word_vectorizer.fit_transform(raw_github_data_filtered['description_plus_topics'])

/Users/dlite/projects/ml_examples/covid19-repo-recommender/covid19-repo-recommender/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [11]:
########################################################
### Build predict function
########################################################

In [12]:
def text_recommender(input_df, word_vectorizer=word_vectorizer,  sparse_vector_matrix = sparse_vector_matrix, repo_lookup=repo_lookup):
    
    input_df['bag_of_words'] = input_df.apply(lambda x: ' '.join(x), axis = 1)
    
    # vectorize the inputted string
    #inputted_vector = word_vectorizer.transform(pd.Series(str(input_string)))
    inputted_vector = word_vectorizer.transform(input_df['bag_of_words'])
    
    # calculate cosine similarity with existing matrix
    one_dimension_cosine_sim = cosine_similarity(inputted_vector, sparse_vector_matrix)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(one_dimension_cosine_sim[0]).sort_values(ascending = False)
    # only show matches that have some similarity
    score_series = score_series[score_series>0]

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # initializing the empty list of recommended repo
    recommended_repos = []
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_repos.append(repo_lookup.loc[i])
        
    return [recommended_repos]

In [13]:
class covid19RepoReco(pyfunc.PythonModel):
   
    ## defining objects needed for leadsModel prediction. 
    def __init__(self,
                 word_vectorizer,
                 sparse_vector_matrix,
                 repo_lookup,
                 text_recommender):
        
        ## Setting up all needed objects
        self.word_vectorizer = word_vectorizer
        self.sparse_vector_matrix = sparse_vector_matrix
        self.repo_lookup = repo_lookup
        self.text_recommender = text_recommender
    
    ## define function with processing and feeding data into prediction at the end
    def predict(self,context,model_input):
        return self.text_recommender(model_input)

In [14]:
# Testing the model
m = covid19RepoReco(word_vectorizer = word_vectorizer,
                                       sparse_vector_matrix = sparse_vector_matrix,
                                       repo_lookup = repo_lookup,
                                       text_recommender = text_recommender)
input = pd.DataFrame([["Python", "Data"]])
m.predict(None,input)

[[github_repo_url                                    https://github.com/mmarchegiani/covid-19
  repo_description                                     data analysis of the covid-19 outbreak
  topics                                                                                     
  owner_repo_name                                                       mmarchegiani/covid-19
  owner_name                                                                     mmarchegiani
  owner_type                                                                             User
  organization_bio                                                                        NaN
  repo_created_day                                                                 2020-03-07
  primary_language_name                                                                Python
  license_name                                                                            NaN
  is_github_pages                                           

In [15]:
########################################################
### Log to MLflow
########################################################

In [16]:
# connect to MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Covid19RepoRecommender") # creates an experiment if it doesn't exist

In [17]:
mlflow_conda_env = {
 'name': 'mlflow-env',
 'channels': ['defaults',
              'conda-forge'],
 'dependencies': ['python=3.7.5',
                  'nltk=3.4.5',
                  'nltk_data',
                  {'pip': ['mlflow==1.6.0',
                           'scikit-learn',
                           'cloudpickle==1.2.2']}
                 ]
}

In [18]:
with mlflow.start_run(run_name="Covid Repo Recommender") as run:
    mlflow.log_param("num_repos_returned", 10)
    
    pyfunc.log_model(
        artifact_path = "covid_repo_reco_pyfunc",
        python_model = covid19RepoReco(word_vectorizer = word_vectorizer,
                                       sparse_vector_matrix = sparse_vector_matrix,
                                       repo_lookup = repo_lookup,
                                       text_recommender = text_recommender),
        conda_env = mlflow_conda_env
    )
    
    run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    
    mlflow.end_run()

In [ ]:
########################################################
### Deploy to Sagemaker
########################################################

In [ ]:
## Note: this requires a MLflow pyfunc docker container to already exist in sagemaker

import mlflow.sagemaker as mfs


# we pull the run and experiment id's from above to create this mlflow location
model_uri = "mlruns/%s/%s/artifacts/covid_repo_reco_pyfunc" % (experiment_id,run_id)

# The region is chosen, pick whats close to you or your systems!
region = "us-east-1"
# The aws account id can be found in the console
aws_account_id = "XXXXXX"
# We use these inputs to automatically reference the sagemaker docker container
image_url = aws_account_id \
            + ".dkr.ecr." \
            + region \
            + ".amazonaws.com/mlflow-pyfunc:1.5.0"

# now we specify the role that we setup for sagemaker in the previous step
sagemaker_arn = "arn:aws:iam::XXXXXX:role/AmazonSageMakerFullAccess"


# finally, we pick a name for our endpoint within sagemaker
endpoint_name = "covid19-repo-rec" 


# with all of the inputs, we run the following to deploy the model it sagemaker
mfs.deploy(app_name=endpoint_name, 
           model_uri=model_uri,
           region_name=region,
           mode="replace", #this should change to replace if the endpoint already exists
           execution_role_arn=sagemaker_arn,
           image_url=image_url, 
           instance_type='ml.t2.medium')